In [4]:
%cd drive/MyDrive/CS273B Final Project

/content/drive/MyDrive/CS273B Final Project


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls

'Attention U-Net Version 2.ipynb'
 dataset
 dataset-20240603T151808Z-001.zip
'dataset U-NET'
 Kaggle_Attention_UNET_Implementation.ipynb
 stage1_solution_combined_masks-20240603T144926Z-001.zip
 stage1_test_data-20240603T145431Z-001.zip
 U-NET_Implementation_Version_2.h5
'U-Net-Implementation Version 2.ipynb'


Unzip dataset files

In [ ]:
import zipfile
import os

# Path to your ZIP file
zip_file_path = 'dataset-20240603T151808Z-001.zip'

# Get the current working directory
current_directory = os.getcwd()

# Extracting the ZIP file in the current working directory
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(current_directory)

In [6]:
import numpy as np
import pandas as pd

import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from tqdm.notebook import tqdm
import time
import random

plt.style.use("dark_background")

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import train_test_split

In [ ]:
BASE_PATH = ""

In [ ]:

# IMG_SIZE = 512


Preparing Data

In [9]:
train_image_dir = "dataset/non-aug/train/images/"
validation_image_dir = "dataset/non-aug/valid/images/"
test_image_dir = "dataset/non-aug/test/images/"

train_mask_dir = "dataset/non-aug/train/masks/"
validation_mask_dir = "dataset/non-aug/valid/masks/"
test_mask_dir = "dataset/non-aug/test/masks/"

In [10]:
train_image_paths = os.listdir(train_image_dir)
for i in range(len(train_image_paths)):
  train_image_paths[i] = train_image_dir + train_image_paths[i]

train_mask_paths = os.listdir(train_mask_dir)
for i in range(len(train_mask_paths)):
  train_mask_paths[i] = train_mask_dir + train_mask_paths[i]

validation_image_paths = os.listdir(validation_image_dir)
for i in range(len(validation_image_paths)):
  validation_image_paths[i] = validation_image_dir + validation_image_paths[i]

validation_mask_paths = os.listdir(validation_mask_dir)
for i in range(len(validation_mask_paths)):
  validation_mask_paths[i] = validation_mask_dir + validation_mask_paths[i]

test_image_paths = os.listdir(test_image_dir)
for i in range(len(test_image_paths)):
  test_image_paths[i] = test_image_dir + test_image_paths[i]

test_mask_paths = os.listdir(test_mask_dir)
for i in range(len(test_mask_paths)):
  test_mask_paths[i] = test_mask_dir + test_mask_paths[i]

In [11]:
sorted_train_image_paths = train_image_paths.copy()
sorted_train_mask_paths = train_mask_paths.copy()
sorted_train_image_paths.sort()
sorted_train_mask_paths.sort()

sorted_val_image_paths = validation_image_paths.copy()
sorted_val_mask_paths = validation_mask_paths.copy()
sorted_val_image_paths.sort()
sorted_val_mask_paths.sort()

sorted_test_image_paths = test_image_paths.copy()
sorted_test_mask_paths = test_mask_paths.copy()
sorted_test_image_paths.sort()
sorted_test_mask_paths.sort()

In [12]:
print(sorted_train_image_paths[0:10])
print(sorted_train_mask_paths[0:10])

['dataset/non-aug/train/images/00071198d059ba7f5914a526d124d28e6d010c92466da21d4a04cd5413362552.png', 'dataset/non-aug/train/images/003cee89357d9fe13516167fd67b609a164651b21934585648c740d2c3d86dc1.png', 'dataset/non-aug/train/images/0121d6759c5adb290c8e828fc882f37dfaf3663ec885c663859948c154a443ed.png', 'dataset/non-aug/train/images/01d44a26f6680c42ba94c9bc6339228579a95d0e2695b149b7cc0c9592b21baf.png', 'dataset/non-aug/train/images/0280fa8f60f6bcae0f97d93c28f60be194f9309ff610dc5845e60455b0f87c21.png', 'dataset/non-aug/train/images/02903040e19ddf92f452907644ad3822918f54af41dd85e5a3fe3e1b6d6f9339.png', 'dataset/non-aug/train/images/03398329ced0c23b9ac3fac84dd53a87d9ffe4d9d10f1b5fe8df8fac12380776.png', 'dataset/non-aug/train/images/03b9306f44e9b8951461623dcbd615550cdcf36ea93b203f2c8fa58ed1dffcbe.png', 'dataset/non-aug/train/images/0402a81e75262469925ea893b6706183832e85324f7b1e08e634129f5d522cdd.png', 'dataset/non-aug/train/images/04acab7636c4cf61d288a5962f15fa456b7bde31a021e5deedfbf51288e4

In [34]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image

class SegmentationDataset(Dataset):
    def __init__(self, image_paths, mask_paths, transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        # Load the image and mask
        image = Image.open(self.image_paths[index]).convert('RGB')
        mask = Image.open(self.mask_paths[index]).convert('L')  # Assuming mask is single channel

        # Apply the transformation
        if self.transform:
            # Transform data using named arguments
            augmented = self.transform(image=np.array(image), mask=np.array(mask))
            image = augmented['image']
            mask = augmented['mask']

        return image, mask


In [14]:
!pip install albumentations==0.4.6


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65153 sha256=d4f945031c37287740b0e46440d004d831aef7d0b5be433327bdd19759106e2a
  Stored in directory: /root/.cache/pip/wheels/f9/d7/0c/6ed42fd872f7d1af78b25045f8b16be330f2c70ae72c83e37d
Successfully built albumentations
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.3.1
    Uninstalling albumentations-1.3.1:
      Successfully uninstalled albumentations-1.3.1


In [35]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

PATCH_SIZE = 128

transforms = A.Compose([
    A.Resize(width = PATCH_SIZE, height = PATCH_SIZE, p=1.0),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Transpose(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.01, scale_limit=0.04, rotate_limit=0, p=0.25),

    A.Normalize(p=1.0),
    ToTensorV2(),

])


In [38]:
train_dataset = SegmentationDataset(train_image_paths, train_mask_paths, transform=transforms)
val_dataset = SegmentationDataset(validation_image_paths, validation_mask_paths, transform=transforms)

In [39]:
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=2)

In [40]:
import torch
torch.cuda.empty_cache()

Attention U-NET

In [21]:
!pip install albumentations==0.4.6


In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import train_test_split

In [42]:
class ConvBlock(nn.Module):
    def __init__(self, ch_in, ch_out):
        super().__init__()
        self.conv = nn.Sequential(
                                  nn.Conv2d(ch_in, ch_out,
                                            kernel_size=3, stride=1,
                                            padding=1, bias=True),
                                  nn.BatchNorm2d(ch_out),
                                  nn.ReLU(inplace=True),
                                  nn.Conv2d(ch_out, ch_out,
                                            kernel_size=3, stride=1,
                                            padding=1, bias=True),
                                  nn.BatchNorm2d(ch_out),
                                  nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x = self.conv(x)
        return x

In [43]:
class UpConvBlock(nn.Module):
    def __init__(self, ch_in, ch_out):
        super().__init__()
        self.up = nn.Sequential(
                                nn.Upsample(scale_factor=2),
                                nn.Conv2d(ch_in, ch_out,
                                         kernel_size=3,stride=1,
                                         padding=1, bias=True),
                                nn.BatchNorm2d(ch_out),
                                nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x = x = self.up(x)
        return x

In [44]:
class AttentionBlock(nn.Module):
    def __init__(self, f_g, f_l, f_int):
        super().__init__()

        self.w_g = nn.Sequential(
                                nn.Conv2d(f_g, f_int,
                                         kernel_size=1, stride=1,
                                         padding=0, bias=True),
                                nn.BatchNorm2d(f_int)
        )

        self.w_x = nn.Sequential(
                                nn.Conv2d(f_l, f_int,
                                         kernel_size=1, stride=1,
                                         padding=0, bias=True),
                                nn.BatchNorm2d(f_int)
        )

        self.psi = nn.Sequential(
                                nn.Conv2d(f_int, 1,
                                         kernel_size=1, stride=1,
                                         padding=0,  bias=True),
                                nn.BatchNorm2d(1),
                                nn.Sigmoid(),
        )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.w_g(g)
        x1 = self.w_x(x)
        psi = self.relu(g1+x1)
        psi = self.psi(psi)

        return psi*x

Model

In [45]:
class AttentionUNet(nn.Module):
    def __init__(self, n_classes=1, in_channel=3, out_channel=1):
        super().__init__()

        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv1 = ConvBlock(ch_in=in_channel, ch_out=64)
        self.conv2 = ConvBlock(ch_in=64, ch_out=128)
        self.conv3 = ConvBlock(ch_in=128, ch_out=256)
        self.conv4 = ConvBlock(ch_in=256, ch_out=512)
        self.conv5 = ConvBlock(ch_in=512, ch_out=1024)

        self.up5 = UpConvBlock(ch_in=1024, ch_out=512)
        self.att5 = AttentionBlock(f_g=512, f_l=512, f_int=256)
        self.upconv5 = ConvBlock(ch_in=1024, ch_out=512)

        self.up4 = UpConvBlock(ch_in=512, ch_out=256)
        self.att4 = AttentionBlock(f_g=256, f_l=256, f_int=128)
        self.upconv4 = ConvBlock(ch_in=512, ch_out=256)

        self.up3 = UpConvBlock(ch_in=256, ch_out=128)
        self.att3 = AttentionBlock(f_g=128, f_l=128, f_int=64)
        self.upconv3 = ConvBlock(ch_in=256, ch_out=128)

        self.up2 = UpConvBlock(ch_in=128, ch_out=64)
        self.att2 = AttentionBlock(f_g=64, f_l=64, f_int=32)
        self.upconv2 = ConvBlock(ch_in=128, ch_out=64)

        self.conv_1x1 = nn.Conv2d(64, out_channel,
                                  kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        # encoder
        x1 = self.conv1(x)

        x2 = self.maxpool(x1)
        x2 = self.conv2(x2)

        x3 = self.maxpool(x2)
        x3 = self.conv3(x3)

        x4 = self.maxpool(x3)
        x4 = self.conv4(x4)

        x5 = self.maxpool(x4)
        x5 = self.conv5(x5)

        # decoder + concat
        d5 = self.up5(x5)
        x4 = self.att5(g=d5, x=x4)
        d5 = torch.concat((x4, d5), dim=1)
        d5 = self.upconv5(d5)

        d4 = self.up4(d5)
        x3 = self.att4(g=d4, x=x3)
        d4 = torch.concat((x3, d4), dim=1)
        d4 = self.upconv4(d4)

        d3 = self.up3(d4)
        x2 = self.att3(g=d3, x=x2)
        d3 = torch.concat((x2, d3), dim=1)
        d3 = self.upconv3(d3)

        d2 = self.up2(d3)
        x1 = self.att2(g=d2, x=x1)
        d2 = torch.concat((x1, d2), dim=1)
        d2 = self.upconv2(d2)

        d1 = self.conv_1x1(d2)

        return d1

In [46]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [47]:
attention_unet = AttentionUNet(n_classes=1).to(device)


In [48]:
output = torch.randn(1,3,128,128).to(device)
output.shape

torch.Size([1, 3, 128, 128])

Segmentation Metric (DICE and IoU)

In [76]:
def dice_coef_metric(predictions, targets, smooth=1e-6):
    predictions = predictions.contiguous().view(-1)
    targets = targets.contiguous().view(-1)

    intersection = (predictions * targets).sum()
    union = predictions.sum() + targets.sum()

    dice = (2. * intersection + smooth) / (union + smooth)
    return dice


In [81]:
def dice_coef_metric(predictions, targets, smooth=1e-6):
    """Calculate the Dice coefficient for NumPy arrays."""
    # Flatten the arrays
    predictions = predictions.flatten()
    targets = targets.flatten()

    intersection = (predictions * targets).sum()
    union = predictions.sum() + targets.sum()

    dice = (2. * intersection + smooth) / (union + smooth)
    return dice


In [77]:
def iou_score(predictions, targets, smooth=1e-6):
    # Flatten label and prediction tensors
    predictions = predictions.contiguous().view(-1)
    targets = targets.contiguous().view(-1)

    intersection = (predictions * targets).sum()
    union = predictions.sum() + targets.sum() - intersection

    iou = (intersection + smooth) / (union + smooth)
    return iou


DICE LOSS

In [65]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True, sigmoid=True):
        super(DiceLoss, self).__init__()
        self.sigmoid = sigmoid

    def forward(self, inputs, targets, smooth=1):
        # Apply sigmoid if inputs are logits
        if self.sigmoid:
            inputs = torch.sigmoid(inputs)

        # Flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice_score = (2. * intersection + smooth) / (inputs.sum() + targets.sum() + smooth)

        return 1 - dice_score


In [59]:
'''

class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        # inputs = F.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return 1 - dice
'''

In [66]:
# sanity check
DiceLoss()(torch.tensor([0.7, 1., 1.]),
              torch.tensor([1.,1.,1.]))

tensor(0.1419)

Training

In [82]:
def train_model(model_name, model, train_loader, val_loader, criterion, optimizer, num_epochs=30):
    for epoch in range(num_epochs):
        model.train()
        train_dice, train_iou = [], []

        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            # Calculate metrics
            output = torch.sigmoid(output)  # Sigmoid to convert outputs to probabilities
            output = (output > 0.5).float()  # Threshold probabilities to get binary mask
            dice = dice_coef_metric(output, target)
            iou = iou_score(output, target)

            train_dice.append(dice.item())
            train_iou.append(iou.item())

        # Validation step
        model.eval()
        with torch.no_grad():
            val_dice, val_iou = [], []
            for data, target in val_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                output = torch.sigmoid(output)  # Sigmoid to convert outputs to probabilities
                output = (output > 0.5).float()  # Threshold probabilities to get binary mask
                dice = dice_coef_metric(output, target)
                iou = iou_score(output, target)

                val_dice.append(dice.item())
                val_iou.append(iou.item())

        print(f"Epoch {epoch+1}, Train Dice: {np.mean(train_dice)}, Train IoU: {np.mean(train_iou)}, Val Dice: {np.mean(val_dice)}, Val IoU: {np.mean(val_iou)}")

In [87]:

def train_model(model_name, model, train_loader, val_loader, train_loss, optimizer, lr_scheduler, num_epochs):
    print(f"[INFO] Model is initializing... {model_name}")

    loss_history = []
    train_history = []
    val_history = []

    for epoch in range(num_epochs):
        model.train()

        losses = []
        train_iou = []

        for i_step, (data, target) in enumerate(tqdm(train_loader)):
            data = data.to(device)
            target = target.to(device)

            outputs = model(data)

            out_cut = np.copy(outputs.data.cpu().numpy())
            out_cut[np.nonzero(out_cut < 0.5)] = 0.0
            out_cut[np.nonzero(out_cut >= 0.5)] = 1.0

            train_dice = dice_coef_metric(out_cut, target.data.cpu().numpy())

            loss = train_loss(outputs, target)

            losses.append(loss.item())
            train_iou.append(train_dice)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        val_mean_iou = compute_iou(model, val_loader)

        loss_history.append(np.array(losses).mean())
        train_history.append(np.array(train_iou).mean())
        val_history.append(val_mean_iou)

        print("Epoch [%d]" % (epoch))
        print("Mean loss on train:", np.array(losses).mean())
        print("Mean IoU on validation:", val_mean_iou)  # Correct label for the output

    return loss_history, train_history, val_history



In [84]:
def compute_iou(model, dataloader):
    """Compute the mean IoU using the provided model and dataloader."""
    total_iou = 0.0
    count = 0

    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        for data, target in dataloader:
            data = data.to(device)
            target = target.to(device)

            outputs = model(data)
            outputs = torch.sigmoid(outputs)  # Convert to probabilities
            preds = (outputs > 0.5).float()  # Binarize predictions

            iou = iou_score(preds, target)
            total_iou += iou.item()
            count += 1

    average_iou = total_iou / count if count != 0 else 0
    return average_iou


In [79]:
opt = torch.optim.Adamax(attention_unet.parameters(), lr=1e-3)


In [88]:
%%time
num_ep = 30
# after 30 does not improve much

aun_lh, aun_th, aun_vh = train_model("Attention UNet", attention_unet, train_dataloader, val_dataloader, DiceLoss(), opt, False, num_ep)
# Assuming you've defined device somewhere, e.g., device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# train_model("Attention UNet", attention_unet, train_dataloader, val_dataloader, DiceLoss(), opt, num_ep)


[INFO] Model is initializing... Attention UNet


  0%|          | 0/268 [00:00<?, ?it/s]

Epoch [0]
Mean loss on train: -0.9143838953616014
Mean IoU on validation: 31.33939555153918


  0%|          | 0/268 [00:00<?, ?it/s]

Epoch [1]
Mean loss on train: -0.9164270642978042
Mean IoU on validation: 31.339543129081157


  0%|          | 0/268 [00:00<?, ?it/s]

KeyboardInterrupt: 